<a href="https://colab.research.google.com/github/juanicolombo97/TP1-OrganizacionDatos/blob/main/TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP1

## Datos Grupo

### Link Github Repository

[Github](https://github.com/juanicolombo97/TP1-OrganizacionDatos)

### Juan Ignacio Colombo


Padron: 103471


Mail: jcolombo@fi.uba.ar

## Imports

In [ ]:
# Import pandas.

import pandas as pd
import seaborn as sns


In [ ]:
# Import files from drive.

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Import datasets.

train_labels_dataset = pd.read_csv("/content/gdrive/MyDrive/1C-2021/Datos/TP1/Datasets/train_labels.csv")
train_values_dataset = pd.read_csv("/content/gdrive/MyDrive/1C-2021/Datos/TP1/Datasets/train_values.csv")


## Analisis Exploratorio

### Analisis inicial.


In [ ]:
train_labels_dataset.head(5)

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [ ]:
train_values_dataset.head(5)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [230]:
# Analizo segun año.
train_values_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  int64 
 1   geo_level_1_id                          260601 non-null  int64 
 2   geo_level_2_id                          260601 non-null  int64 
 3   geo_level_3_id                          260601 non-null  int64 
 4   count_floors_pre_eq                     260601 non-null  int64 
 5   age                                     260601 non-null  int64 
 6   area_percentage                         260601 non-null  int64 
 7   height_percentage                       260601 non-null  int64 
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 

### Manejo Nulos y Nan.

In [233]:
cantidad_edificios = train_values_dataset["building_id"].count()
cantidad_edificios

260601

In [237]:
# Cantidad de nulos por columna.
train_values_dataset.isnull().sum()

building_id                               0
geo_level_1_id                            0
geo_level_2_id                            0
geo_level_3_id                            0
count_floors_pre_eq                       0
age                                       0
area_percentage                           0
height_percentage                         0
land_surface_condition                    0
foundation_type                           0
roof_type                                 0
ground_floor_type                         0
other_floor_type                          0
position                                  0
plan_configuration                        0
has_superstructure_adobe_mud              0
has_superstructure_mud_mortar_stone       0
has_superstructure_stone_flag             0
has_superstructure_cement_mortar_stone    0
has_superstructure_mud_mortar_brick       0
has_superstructure_cement_mortar_brick    0
has_superstructure_timber                 0
has_superstructure_bamboo       

In [238]:
# Cantidad de Nan por columna.
train_values_dataset.isna().sum()

building_id                               0
geo_level_1_id                            0
geo_level_2_id                            0
geo_level_3_id                            0
count_floors_pre_eq                       0
age                                       0
area_percentage                           0
height_percentage                         0
land_surface_condition                    0
foundation_type                           0
roof_type                                 0
ground_floor_type                         0
other_floor_type                          0
position                                  0
plan_configuration                        0
has_superstructure_adobe_mud              0
has_superstructure_mud_mortar_stone       0
has_superstructure_stone_flag             0
has_superstructure_cement_mortar_stone    0
has_superstructure_mud_mortar_brick       0
has_superstructure_cement_mortar_brick    0
has_superstructure_timber                 0
has_superstructure_bamboo       

### Porcentaje datos validos.

In [252]:
datos_validos = train_values_dataset.count().reset_index()
datos_validos["Porcentaje_datos_validos"] = (cantidad_edificios / datos_validos[0]) * 100
datos_validos_por_columna = datos_validos[["index","Porcentaje_datos_validos"]]
datos_validos_por_columna.to_csv("/content/gdrive/MyDrive/1C-2021/Datos/TP1/CSVS/datos_validos_por_columna.csv")

### Analisis Segun Edad

#### Filtrado datos y merge.

In [272]:
datos_edad = train_values_dataset[["building_id", "age"]]
datos_edad_merged = pd.merge(left=datos_edad, right=train_labels_dataset, how="inner", on="building_id")
datos_edad_filtrados = datos_edad_merged[["damage_grade", "age"]]
datos_edad_filtrados.head(1)

,damage_grade,age
0,3,30


#### Edades comparadas a promedio segun damage_grade.

In [285]:
datos_promedio = datos_edad_filtrados
datos_promedio["Supera_Promedio_Edad"] = datos_promedio["age"] > datos_promedio["age"].mean()
datos_promedio["Debajo_Promedio_Edad"] = datos_promedio["age"] <= datos_promedio["age"].mean()
datos_promedio_final = datos_promedio.groupby("damage_grade").agg({"Supera_Promedio_Edad": "count", "Debajo_Promedio_Edad": "count"}).reset_index()
datos_promedio_final.to_csv("/content/gdrive/MyDrive/1C-2021/Datos/TP1/CSVS/datos_promedio_final.csv") 

#### Rango edades segun damage_grade.

In [284]:
# Obtengo los rangos de edad segun el damage grade, para edificios entre 0 a 100 anios.
datos_rangos = datos_edad_filtrados
datos_rangos["Damage_3"] = datos_rangos["damage_grade"] == 3
datos_rangos["Damage_2"] = datos_rangos["damage_grade"] == 2
datos_rangos["Damage_1"] = datos_rangos["damage_grade"] == 1
datos_rangos_final = datos_rangos.groupby("age").agg({"Damage_3" : "sum", "Damage_2" : "sum", "Damage_1" : "sum"}).reset_index()
datos_rangos_edad_menor_100 = datos_rangos_final[datos_rangos_final["age"] < 100]
datos_rangos_edad_menor_100.to_csv("/content/gdrive/MyDrive/1C-2021/Datos/TP1/CSVS/datos_rangos_edad_menor_100.csv") 